In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
#pio.renderers.default = 'colab'   # try changing this in case your plots aren't shown
import numpy as np
from plotly.subplots import make_subplots
import json
import geojson

In [ ]:
df=pd.read_csv("/Users/rajendravrk/SIT/Migros-challenge/Migros-Challenge/data/processed/supermarket3.csv")

In [ ]:
df.shape

In [ ]:
del df['Unnamed: 0']

In [ ]:
del df['Unnamed: 0.1']

In [ ]:
df.head()

In [ ]:
df_n=df['formatted_address'].str.split(',', expand=True)

In [ ]:
del df_n[3]

In [ ]:
del df_n[4]

In [ ]:
for i in df_n.iloc:
    if i[2]== 'None' :
        i[2]='Switzerland'

In [ ]:
#df.columns = ['new_col1', 'new_col2', 'new_col3', 'new_col4']

df_n.columns = ['Street', 'commune','country']
df_n

In [ ]:
result = pd.concat([df, df_n], axis=1, join='inner')
result

In [ ]:
del result['price_level']

In [ ]:
result.head()

In [ ]:
f = open('/Users/rajendravrk/SIT/Migros-challenge/Migros-Challenge/data/raw/countries.geojson')

# returns JSON object as 
# a dictionary
data = json.load(f)

In [ ]:
result.head()

In [ ]:
len(result.name.unique())

In [ ]:
stores=['aldi', 'migro' , 'lidl', 'coop']

In [ ]:
def get_store_name(x):
    x=str(x)
    x=x.lower()
    for store in stores:
        if store in x:
            return store
            

In [ ]:
result['store_name'] = result['name'].apply(get_store_name)

In [ ]:
result=result.dropna(subset='store_name')

In [ ]:
result.head(15)

### These is data regarding plotting of 4 big supermarkets in switzerland

In [ ]:
result.to_csv('/Users/rajendravrk/SIT/my-first-streamlitapp/Migros\ Challenge/data/processed/supermarkets_new.csv')

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(result, lat='geometry.location.lat', lon='geometry.location.lng', hover_name="name",
                        zoom=7, height=30, color= 'store_name')
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, height=800)
fig.show()

## 1. Approach through Macro economic Factors for new store location

In [ ]:
bip_kanton = pd.read_csv('/Users/rajendravrk/SIT/Migros-challenge/Migros-Challenge/data/processed/BIP_Kanton_2019.csv')
population_df = pd.read_csv('/Users/rajendravrk/SIT/Migros-challenge/Migros-Challenge/data/processed/df_population_cleaned_final.csv')
income_zürich_df = pd.read_csv('/Users/rajendravrk/SIT/Migros-challenge/Migros-Challenge/data/processed/income_zurich_2021.csv')
# Open the Geojson, and change the key for the maps - BFS number, works not with a point
with open("/Users/rajendravrk/SIT/Migros-challenge/Migros-Challenge/data/raw/gemeinden-geojson.geojson") as f:
    gj = geojson.load(f)

for i in range(len(gj["features"])):
    gj["features"][i]['properties']['gemeinde_BFS_NUMMER'] = gj["features"][i]['properties'].pop('gemeinde.BFS_NUMMER')



In [ ]:
population_df.head(10)

In [ ]:
#Change of the poulation in the Communes in percent, use the csv df_population_cleaned_final.csv

fig_pop_change = go.Figure(go.Choroplethmapbox(geojson=gj, locations=population_df.Number_of_commune, z=population_df.Population_change_perc,
                                    colorscale="Viridis",featureidkey="properties.gemeinde_BFS_NUMMER",
                                    zmin = -20 , zmax = 30, marker_opacity=0.5, marker_line_width=0))

fig_pop_change.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=6, mapbox_center =  {"lat": 46.8200, "lon": 8.4070} )

fig_pop_change.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_pop_change.show()


In [ ]:
# Show the average income for the Kanton of Zürich, use the csv income_zürich_2021.csv

fig_inc = go.Figure(go.Choroplethmapbox(geojson=gj, locations=income_zürich_df.BFS_NR, z=income_zürich_df.INDIKATOR_VALUE,
                                    colorscale="Viridis",featureidkey="properties.gemeinde_BFS_NUMMER",
                                    zmin = 60000 , zmax = 100000, marker_opacity=0.5, marker_line_width=0))

fig_inc.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=6, mapbox_center = {"lat": 47.3769, "lon": 8.5417})

fig_inc.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_inc.show()


In [ ]:
#GDP per Kantone, use the csv BIP_Kanton_2019.csv

bip_kanton = bip_kanton.sort_values(by='GDP_2019',ascending=False)

fig_gdp = px.bar(bip_kanton, x='Kanton_name', y='GDP_2019', labels={
                     "Kanton_name": "Name of the Kantones",
                     "GDP_2019": "GDP in millions (CHF)"})

fig_gdp.update_layout(title = 'GDP in millions (CFH) per Kanton in 2019', title_x=0.5)
fig_gdp.show()

## 2. Approach through presence of other stores in each Commune

In [ ]:
merged_df_only_relevant_big_supermarkets = pd.read_csv("/Users/rajendravrk/SIT/Migros-challenge/Migros-Challenge/data/processed/merged_df_only_relevant_big_supermarkets.csv")


In [ ]:
del merged_df_only_relevant_big_supermarkets['Unnamed: 0']

In [ ]:
del merged_df_only_relevant_big_supermarkets['Unnamed: 0.1']

In [ ]:
merged_df_only_relevant_big_supermarkets.head(1)

In [ ]:
# showing only the N of Supermarkets per 1000 residents

fig = go.Figure(go.Choroplethmapbox(geojson=gj, locations=merged_df_only_relevant_big_supermarkets.Number_of_commune, z=merged_df_only_relevant_big_supermarkets.N_markets_per_1k_residents,
                                    colorscale="RdBu",featureidkey="properties.gemeinde_BFS_NUMMER",
                                    #text=pop_clean['text'],
                                    zmin = 0 , zmax = 10, marker_opacity=0.5, marker_line_width=0))
                
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=6, mapbox_center = {"lat": 46.8200, "lon": 8.4070})

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# showing only the big chain Supermarkets per 1000 residents


fig = go.Figure(go.Choroplethmapbox(geojson=gj, locations=merged_df_only_relevant_big_supermarkets.Number_of_commune, z=merged_df_only_relevant_big_supermarkets.N_big_markets_per_1k_residents,
                                    colorscale="Viridis",featureidkey="properties.gemeinde_BFS_NUMMER",
                                    #text=pop_clean['text'],
                                    zmin = 0 , zmax = 1, marker_opacity=0.5, marker_line_width=0))
                
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=6, mapbox_center = {"lat": 46.8200, "lon": 8.4070})

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})



fig.show()